In [1]:
# Import the necessary libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [20]:
# Set Up Database Connection
from sqlalchemy import create_engine

# details
import os

username = os.getenv('DB_USERNAME')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
database = os.getenv('DB_NAME')

engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}/{database}')

In [3]:
# Define the subreddit you want to scrape and the URL

url = f'https://www.reddit.com/r/television/search/?q=Paramount+&restrict_sr=1'

# Set up the header with a User-Agent
headers = {'User-Agent': 'Mozilla/5.0'}

In [4]:
# Send the GET request to the subreddit URL
response = requests.get(url, headers=headers)

In [5]:
response

<Response [200]>

In [6]:
response.text

'\n    <!DOCTYPE html>\n    <html lang="en-US" class="is-search-results-page theme-beta">\n      <head prefix="og: https://ogp.me/ns#">\n        <title>Reddit - Dive into anything</title>\n        <meta name="viewport" content="width=device-width, initial-scale=1, viewport-fit=cover">\n        <meta name="apple-mobile-web-app-capable" content="yes">\n        <meta name="apple-mobile-web-app-status-bar-style" content="black">\n        <meta name="msapplication-navbutton-color" content="#000000">\n        <meta name="theme-color" content="#000000">\n        \n        \n        \n        <link rel="preconnect" href="https://www.redditstatic.com/" crossorigin="anonymous" />\n        <link rel="dns-prefetch" href="https://www.redditstatic.com/" />\n      \n        \n    <script type="module" nonce="H+hPLQg77VQ4IRnvfXYYYw==">!function(e,n,o,t,i,r,a){var d=!1,c=!1,f=[],s=function(e){("e"in e||"p"in e||e.f&&e.f.indexOf("capture")>-1||e.f&&e.f.indexOf("withScope")>-1||e.f&&e.f.indexOf("showRepo

In [7]:
soup = BeautifulSoup(response.text, 'html.parser')

In [21]:
soup.prettify()

'<!DOCTYPE html>\n<html class="is-search-results-page theme-beta" lang="en-US">\n <head prefix="og: https://ogp.me/ns#">\n  <title>\n   Reddit - Dive into anything\n  </title>\n  <meta content="width=device-width, initial-scale=1, viewport-fit=cover" name="viewport"/>\n  <meta content="yes" name="apple-mobile-web-app-capable"/>\n  <meta content="black" name="apple-mobile-web-app-status-bar-style"/>\n  <meta content="#000000" name="msapplication-navbutton-color"/>\n  <meta content="#000000" name="theme-color"/>\n  <link crossorigin="anonymous" href="https://www.redditstatic.com/" rel="preconnect"/>\n  <link href="https://www.redditstatic.com/" rel="dns-prefetch"/>\n  <script nonce="H+hPLQg77VQ4IRnvfXYYYw==" type="module">\n   !function(e,n,o,t,i,r,a){var d=!1,c=!1,f=[],s=function(e){("e"in e||"p"in e||e.f&&e.f.indexOf("capture")>-1||e.f&&e.f.indexOf("withScope")>-1||e.f&&e.f.indexOf("showReportDialog")>-1)&&u(f),s.data.push(e)};function u(n){function o(){import("https://www.redditstatic

In [29]:
# Data extraction using the specific attribute 'data-testid'
links = soup.findAll('a', {'data-testid': 'post-title'})

In [30]:
# Extract title and href from the links
data = []
for link in links:
    title = link.text.strip()
    href = link['href']

    # Append the extracted information to the data list
    data.append({
        'title': title,
        'link': href
    })

In [33]:
#data

In [32]:
# Create DataFrame
df = pd.DataFrame(data)

In [34]:
# Check DataFrame contents
print("Extracted Data:")
print(df.head())  # Print the first few rows to verify the data


Extracted Data:
                                               title  \
0       Sony in Talks to Join a Bid to Buy Paramount   
1  Paramount Plus is legit the worst streaming se...   
2  Comcast, Paramount In Talks to Combine Peacock...   
3  Paramount as We Know It Has 2 Years Left, at M...   
4  Paramount Adds 6.3M Global Streaming Subs in F...   

                                                link  
0  /r/television/comments/1c7gyo1/sony_in_talks_t...  
1  /r/television/comments/197nbvc/paramount_plus_...  
2  /r/television/comments/1aselow/comcast_paramou...  
3  /r/television/comments/14gv55p/paramount_as_we...  
4  /r/television/comments/uhdmfc/paramount_adds_6...  


In [35]:
# Set up SQLite connection
engine = create_engine('sqlite:///reddit_data.db')  # This line sets up a connection to a SQLite database named reddit_data.db

In [36]:
# Load data into the SQLite database
df.to_sql('reddit_posts', con=engine, if_exists='replace', index=False)

print("Data successfully loaded into SQLite database.")

Data successfully loaded into SQLite database.
